In [1]:
print("Akish")

Akish


In [1]:
rail_str = """
<rail version="0.1">
<output>
   <string
       name="generated_sql"
       description="Generate SQL for the given natural language instruction."
       format="bug-free-sql"
       on-fail-bug-free-sql="reask" 
   />
</output>

<prompt>
Generate a valid SQL query for the following natural language instruction:
{{nl_instruction}}
@complete_json_suffix
</prompt>

</rail>
"""

In [2]:
import guardrails as gd
from rich import print

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\akish.pothuri\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
c:\Users\akish.pothuri\python\LangChain\SQl_Script_chatbot\chatbot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
guard = gd.Guard.from_rail_string(rail_str)